In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.optimizers import SGD
from keras.utils import np_utils
from imutils import paths
import numpy as np
import argparse
import cv2
import os, sys
import pandas as pd
from pathlib import Path

/home/akhiyar/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
row = 128
col = 128
channel = 3
def image_to_feature_vector(image, size=(row, col)):
    
    #return cv2.resize(image, size).flatten()
    return cv2.resize(image, size)

In [3]:
# hah = '../dataset/arara-classify/eomf/croplands/2017-07-09_11-11-55_IMG_20170706_102252.jpg'
# hah.split("/")[4]

In [4]:
# #data = []
# labels = []

# print ("[INFO] describing images...")
# imagePaths = list(paths.list_images('../dataset/arara-classify/eomf/'))

# count = len(imagePaths)
# data = np.ndarray((count, channel, row, col), dtype=np.uint8)

# for i, imagePath in enumerate(imagePaths):
    
#     imagePath = str(Path(imagePath))
#     #print(imagePath)
#     image = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
#     #image = cv2.imread(imagePath)
#     if image is None:
#         continue
        
#     label = imagePath.split("/")[4]

#     features = image_to_feature_vector(image)
#     #data.append(features)
#     data[i] = features
#     labels.append(label)

#     if i > 0 and i % 100 == 0:
#         print ("[INFO] processed {}/{}".format(i, len(imagePaths)))



In [5]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for i, image in enumerate(list_of_images):
        img = cv2.imread(image)
        if img is None:
            continue
        x.append(cv2.resize(img, (row, col), interpolation=cv2.INTER_CUBIC))
    

        label = image.split("/")[4]
        y.append(label)
        if i > 0 and i % 100 == 0:
            print ("[INFO] processed {}/{}".format(i, len(list_of_images)))
    
    return x, y

imagePaths = list(paths.list_images('../dataset/arara-classify/eomf/'))
data, labels = prepare_data(imagePaths)

[INFO] processed 100/714
[INFO] processed 200/714
[INFO] processed 300/714
[INFO] processed 400/714
[INFO] processed 500/714
[INFO] processed 600/714
[INFO] processed 700/714


In [6]:
data = np.array(data)
labels = np.array(labels)

In [7]:
print(len(data), data.shape)
print(len(labels))

647 (647, 128, 128, 3)
647


In [8]:
data = data[0:len(labels)]
print(len(data), data.shape)
print(len(labels))
processed = len(labels)

647 (647, 128, 128, 3)
647


In [9]:
# data = data.reshape(1, processed*channel*row*col)
# le = LabelEncoder()
# labels = le.fit_transform(labels)
# labels = np_utils.to_categorical(labels, 4)

In [10]:
# np.savetxt("processed_data.csv", data, delimiter=";")
# np.savetxt("processed_labes.csv", labels, delimiter=";")

In [11]:
# le = LabelEncoder()
# labels = le.fit_transform(labels)

# data = np.array(data) / 255.0
# labels = np_utils.to_categorical(labels, 4)

# print ("[INFO] saving processed data into csv files...")
# np.savetxt("processed_data.csv", data, delimiter=";")
# np.savetxt("processed_labes.csv", labels, delimiter=";")

In [12]:
le = LabelEncoder()
labels = le.fit_transform(labels)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.20, random_state=42)

In [13]:
X_train = X_train.reshape(X_train.shape[0], row, col, channel).astype('float32')
X_valid = X_valid.reshape(X_valid.shape[0], row, col, channel).astype('float32')

# # normalize inputs from 0-255 to 0-1
# X_train = X_train / 255
# X_valid = X_valid / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)
num_classes = y_valid.shape[1]
num_classes

4

In [18]:
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# def larger_model():
# 	# create model
# 	model = Sequential()
# 	model.add(Convolution2D(64, 5, 5, border_mode='valid', input_shape=(row, col, channel), activation='relu'))
# 	model.add(MaxPooling2D(pool_size=(2, 2)))
# 	model.add(Convolution2D(32, 3, 3, activation='relu'))
# 	model.add(MaxPooling2D(pool_size=(2, 2)))
# 	model.add(Dropout(0.2))
# 	model.add(Flatten())
# 	model.add(Dense(128, activation='relu'))
# 	model.add(Dense(50, activation='relu'))
# 	model.add(Dense(num_classes, activation='softmax'))
# 	# Compile model
# 	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	return model

In [22]:
def larger_model():
    # Set the CNN model
    # my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

    model = Sequential()

    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',
                     activation ='relu', input_shape = (row,col,channel)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',
                     activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation = "softmax"))

    # Define the optimizer
    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

    # Compile the model
    model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

    return model

In [23]:
# # build the model
# model = larger_model()
# # Fit the model
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), nb_epoch=100, batch_size=250, verbose=2)
# # Final evaluation of the model
# scores = model.evaluate(X_valid, y_valid, verbose=0)
# print("Classification Error: %.2f%%" % (100-scores[1]*100))

In [24]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

epochs = 100 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86


datagen = ImageDataGenerator(
        rescale=1. / 255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

# Fit the model
history = larger_model().fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_valid, y_valid),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/100
 - 9s - loss: 7.4519 - acc: 0.2232 - val_loss: 12.2745 - val_acc: 0.2385
Epoch 2/100
 - 3s - loss: 1.5383 - acc: 0.2252 - val_loss: 5.1922 - val_acc: 0.2615
Epoch 3/100
 - 4s - loss: 1.3845 - acc: 0.3723 - val_loss: 11.1690 - val_acc: 0.2846
Epoch 4/100
 - 4s - loss: 1.3918 - acc: 0.2192 - val_loss: 9.0176 - val_acc: 0.2462
Epoch 5/100
 - 3s - loss: 1.3815 - acc: 0.2946 - val_loss: 8.5907 - val_acc: 0.3769
Epoch 6/100
 - 3s - loss: 1.3727 - acc: 0.2471 - val_loss: 9.7868 - val_acc: 0.3538
Epoch 7/100
 - 3s - loss: 1.4547 - acc: 0.3783 - val_loss: 1.4877 - val_acc: 0.2308
Epoch 8/100
 - 3s - loss: 1.3773 - acc: 0.3823 - val_loss: 12.0998 - val_acc: 0.2462
Epoch 9/100
 - 3s - loss: 1.3890 - acc: 0.2558 - val_loss: 1.8715 - val_acc: 0.2769

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 10/100
 - 3s - loss: 1.3867 - acc: 0.2411 - val_loss: 4.4047 - val_acc: 0.3385
Epoch 11/100
 - 3s - loss: 1.3900 - acc: 0.3108 - val_loss: 7.1637 - val_ac

Epoch 91/100
 - 3s - loss: 1.0455 - acc: 0.5875 - val_loss: 8.7349 - val_acc: 0.4462
Epoch 92/100
 - 3s - loss: 1.1564 - acc: 0.4463 - val_loss: 8.7169 - val_acc: 0.4462
Epoch 93/100
 - 3s - loss: 1.0299 - acc: 0.6114 - val_loss: 8.5142 - val_acc: 0.4538
Epoch 94/100
 - 3s - loss: 1.2613 - acc: 0.4603 - val_loss: 8.6685 - val_acc: 0.4462
Epoch 95/100
 - 3s - loss: 1.1276 - acc: 0.5174 - val_loss: 8.6640 - val_acc: 0.4385
Epoch 96/100
 - 3s - loss: 0.9725 - acc: 0.6393 - val_loss: 8.7995 - val_acc: 0.4385
Epoch 97/100
 - 3s - loss: 1.0867 - acc: 0.5816 - val_loss: 8.6061 - val_acc: 0.4538
Epoch 98/100
 - 3s - loss: 1.0878 - acc: 0.6055 - val_loss: 8.5725 - val_acc: 0.4462
Epoch 99/100
 - 3s - loss: 1.0295 - acc: 0.5915 - val_loss: 8.6404 - val_acc: 0.4308
Epoch 100/100
 - 3s - loss: 1.0685 - acc: 0.5835 - val_loss: 8.6225 - val_acc: 0.4308
